# INTRUSION DETECTION MODEL

## Import Modules

In [1]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt, seaborn as sns
%matplotlib inline

## Load Dataset

In [10]:
train_df = pd.read_csv("Data/KDDSelectedTrain.csv")
test_df = pd.read_csv("Data/KDDSelectedTest.csv")

In [11]:
train_df.head()

,is_intrusion,flag_SF,dst_host_same_srv_rate,logged_in,serror_rate,count,service_http,service_private,dst_host_count,service_domain_u,srv_rerror_rate
0,0,1,0.00,0,0.0,13,0,0,255,0,0.0
1,1,0,0.10,0,1.0,123,0,1,255,0,0.0
2,0,1,1.00,1,0.2,5,1,0,30,0,0.0
3,0,1,1.00,1,0.0,30,1,0,255,0,0.0
4,1,0,0.07,0,0.0,121,0,1,255,0,1.0


In [12]:
test_df.head()

,is_intrusion,flag_SF,dst_host_same_srv_rate,logged_in,serror_rate,count,service_http,service_private,dst_host_count,service_domain_u,srv_rerror_rate
0,1,0,0.00,0,0.0,136,0,1,255,0,1.0
1,0,1,0.61,0,0.0,1,0,0,134,0,0.0
2,1,1,1.00,0,0.0,1,0,0,3,0,0.0
3,1,0,0.31,0,0.0,1,0,0,29,0,0.5
4,0,1,1.00,1,0.0,4,1,0,155,0,0.0


## Create Feature Set (Data Matrix X) and Target (1D Vector y)

In [13]:
y_train = train_df['is_intrusion']
X_train = train_df.drop(columns = 'is_intrusion')

y_test = test_df['is_intrusion']
X_test = test_df.drop(columns = 'is_intrusion')

## Standardize Data

In [14]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)

scaler = StandardScaler().fit(X_test)
X_test = scaler.transform(X_test)

## Benchmarking Classifiers

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC 

classifier_list = ['Logistic Regression', 'K Nearest Neighbors', 'Decision Tree', 'SVC']

classifiers = [
    LogisticRegression(n_jobs=-1, random_state=42),
    KNeighborsClassifier(),
    DecisionTreeClassifier(random_state=42),
    SVC(kernel= 'rbf', random_state=42)
]